In [2]:
import ee
import geemap
import pandas as pd
import geopandas as gpd

ee.Initialize()

def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi


def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])


def union_polygons(poly_ls):
    basePoly = poly_ls[0]
    for poly in poly_ls[1:]:
        basePoly = basePoly.union(poly, maxError = 1)

    return basePoly

In [3]:
county_list = [
    'Cherokee', 'Clayton', 'Cobb', 
    'DeKalb', 'Douglas', 'Fayette', 
    'Forsyth', 'Fulton', 'Gwinnett', 
    'Henry','Rockdale'
]

ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
poly_list = []

for county in county_list:
    county_shp = ga_counties[ga_counties["NAME10"] == county].geometry.values[0]
    poly_list.append(pd_shp_to_ee_poly(county_shp).simplify(maxError = 1))

atlanta = union_polygons(poly_list).simplify(maxError = 1)

atlanta_city = ee.Geometry.Polygon(
    [[-84.8748779, 34.4001112],
    [-84.8941040, 33.2869160],
    [-83.4439087, 33.3052817],
    [-83.4878540, 34.4159734]], None, False
)

In [4]:
# Image of georgia counties
# TODO: Figure out how to make these boundaries white...
outline = ee.Image().paint(
    featureCollection = ee.FeatureCollection(poly_list), 
    color = 0,
    width = 1
)

In [5]:
def landsat_atlanta(bands, fname):
    collection = geemap.landsat_timeseries(
        roi = atlanta, 
        start_year = 1985, end_year = 2019, 
        start_date = '06-10', end_date = '09-20'
    ).map(lambda image: image.blend(outline))

    video_args = {
        'dimensions': 768,
        'region': atlanta,
        'framesPerSecond': 3,
        'bands': bands,
        'min': 0,
        'max': 4000,
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 1985, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )


def naip_atlanta(fname):
    collection = geemap.naip_timeseries(
        roi = atlanta,
        start_year = 2003, end_year = 2021
    )

    video_args = {
        'dimensions': 768,
        'region': atlanta,
        'framesPerSecond': 2,
        'min': 0.0,
        'max': 255.0,
        'bands': ['R', 'G', 'B'],
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 2003, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )


def landsat_roi(roi, bands, fname):
    collection = geemap.landsat_timeseries(
        roi = roi, 
        start_year = 1985, end_year = 2021, 
        start_date = '06-10', end_date = '09-20'
    )

    video_args = {
        'dimensions': 768,
        'region': roi,
        'framesPerSecond': 2,
        'bands': bands,
        'min': 0,
        'max': 4000,
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 1985, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )

In [15]:
## Visualizing Atlanta since 1985
bands = ['Red', 'Green', 'Blue']
fname = 'vids/georgia-atlanta.gif'
landsat_atlanta(bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-atlanta.gif


Output()

![](vids/georgia-atlanta.gif)

In [6]:
cherokee, _ = get_county_roi('Cherokee')
bands = ['Red', 'Green', 'Blue']
fname = 'vids/cherokee-landsat.gif'
landsat_roi(cherokee, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/cherokee-landsat.gif


![](vids/cherokee-landsat.gif)

In [8]:
fname = 'vids/georgia-atlanta-naip.gif'
naip_atlanta(fname)
geemap.show_image(fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-atlanta-naip.gif


Output()

![](vids/georgia-atlanta-naip.gif)

In [10]:
#geemap.sentinel2_timelapse(
#    roi = atlanta,
#    out_gif = 'vids/georgia-atlanta-sentinel2.gif',
#    start_year = 2015,
#    end_year = 2021
#)
# Can also consider using sentinel images at some point
#geemap.show_image('vids/georgia-atlanta-sentinel2.gif')

In [16]:
bands = ['SWIR1', 'NIR', 'Blue']
fname = 'vids/georgia-atlanta-agriculture.gif'
landsat_atlanta(bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-atlanta-agriculture.gif


Output()

![](vids/georgia-atlanta-agriculture.gif)

In [17]:
bands = ['NIR', 'Red', 'Green']
fname = 'vids/georgia-atlanta-vegeation.gif'
landsat_atlanta(bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-vegeation.gif


Output()

![](vids/georgia-ts-vegeation.gif)

In [6]:
bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/georgia-atlanta-geology.gif'
landsat_atlanta(bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-atlanta-geology.gif


Output()

![](vids/georgia-atlanta-geology.gif)

In [4]:
collection_virrs = geemap.create_timeseries(
    collection = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG",
    region = atlanta, 
    frequency = 'month',
    start_date = '2012-04-01', 
    end_date = '2021-05-01'
)

video_args_virrs = {
    'dimensions': 500,
    'region': atlanta,
    'framesPerSecond': 5,
    'bands': ['avg_rad'],
    'min': 0,
    'max': 60,
    'bestEffort': True,
    'palette': ['000000', 'FFFFFF']
}

fname = 'vids/atlanta-nighttime.gif'

dates_virrs = pd.date_range(
    start = "2012-04", end = "2021-05", freq = "M"
    ).strftime('%Y-%m').tolist()

geemap.download_ee_video(
    collection_virrs, video_args_virrs, fname
)

geemap.add_text_to_gif(
    fname, fname, xy=('85%', '3%'), 
    text_sequence = dates_virrs[:-1], font_size = 20, 
    font_color = '#ffffff', 
    add_progress_bar = False
)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/atlanta-nighttime.gif


Output()

![](vids/atlanta-nighttime.gif)

In [9]:
bands = ['Red', 'Green', 'Blue']
fname = 'vids/georgia-atlanta-city.gif'
landsat_roi(atlanta_city, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-atlanta-city.gif


![](vids/georgia-atlanta-city.gif)
![](vids/georgia-atlanta-city-geology.gif)